In [1]:
import pandas as pd
import numpy as np
import pymc as pm

C:\Users\can2hr\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
WARNING (aesara.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (aesara.configdefaults): g++ not detected!  Aesara will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set Aesara flags cxx to an empty string.
WARNING (aesara.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
C:\Users\can2hr\Anaconda3\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: 4.0.0-unsupported is an invalid version and will not be supported in a future release
  warnings.warn(


## Speed
Our response will be predicted get off time
    - Need to decide how we want to do this
    - Option 1 is amount of time to get to a certain distance (Say 1.5 yards)
    - Option 2 would be predicted yards given a cetain amount of time
For predictors:
    - Height/weight of the tackle
    - Quarter
    - Whether it's an obvious passing situation
    - Left vs right tackle?

## Power
Our response here is some variation of percentage of the distance to the QB given up once engaged with the defensive end
For predictors:
    - Height/weight of both the tackle and the end
    - Obvious passing situation
    - Time to throw for the QB
    - Quarter

In [2]:
model_data = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/clustered.csv')

In [3]:
model_data.columns

Index(['uniqueplayId', 'QB_Flip', 'rushType', 'Power_prob', 'Speed_prob',
       'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'playDirection', 'event', 'ball_snap_frame', 'end_frame',
       'play_length_frames', 'Right_end', 'Left_end', 'QB_nflId',
       'QB_jerseyNumber', 'QB_team', 'QB_x', 'QB_y', 'QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp',
       'QB_pff_nflIdBlockedPlayer', 'QB_pff_blockType', 'QB_new_x', 'QB_new_y',
       'ball_nflId', 'ball_jerseyNumber', 'ball_team', 'ball_x', 'ball_y',
       'ball_s', 'ball_a', 'ball_dis', 'ball_o', 'ball_dir', 'ball_pff_role',
       'ball_pff_positionLinedUp', 'ball_pff_nflIdBlockedPlayer',
       'ball_pff_blockType', 'ball_new_x', 'ball_new_y', 'ball_Flip',
       'Tackle_nflId', 'Tackle_jerseyNumber', 'Tackle_team', 'Tackle_x',
       'Tackle_y', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLined

In [39]:
model_data2 = model_data[['uniqueplayId','QB_Flip','rushType','Power_prob','Speed_prob',
            'football_x', 'football_y', 'gameId', 'playId', 'frameId', 'time',
       'event','play_length_frames','QB_s', 'QB_a', 'QB_dis',
       'QB_o', 'QB_dir', 'QB_pff_role', 'QB_pff_positionLinedUp','QB_new_x', 'QB_new_y',
            'ball_new_x', 'ball_new_y', 'ball_Flip','ball_snap_frame',
       'Tackle_nflId', 'Tackle_s', 'Tackle_a', 'Tackle_dis', 'Tackle_o',
       'Tackle_dir', 'Tackle_pff_role', 'Tackle_pff_positionLinedUp',
       'Tackle_pff_nflIdBlockedPlayer', 'Tackle_pff_blockType', 'Tackle_new_x',
       'Tackle_new_y', 'Tackle_Flip', 'End_nflId', 'End_s', 'End_a', 'End_dis', 'End_o',
       'End_dir', 'End_pff_role', 'End_pff_positionLinedUp', 'End_new_x',
       'End_new_y', 'End_Flip', 'tackle_end_dist', 'tackle_end_facing']]

In [40]:
players = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/players.csv')

In [41]:
players = players[['height', 'weight', 'nflId','displayName']]

In [42]:
model_data3 = model_data2.merge(players, left_on = 'Tackle_nflId', right_on = 'nflId', how = 'left')

In [43]:
model_data4 = model_data3.rename(columns = {'height':'tackle_height','weight':'tackle_weight','displayName':'tackle_name'}).drop('nflId', axis = 1)

In [44]:
model_data5 = model_data4.merge(players, left_on = 'End_nflId', right_on = 'nflId', how = 'left')

In [45]:
model_data6 = model_data5.rename(columns = {'height':'end_height','weight':'end_weight','displayName':'end_name'}).drop('nflId', axis = 1)

In [46]:
plays = pd.read_csv('https://media.githubusercontent.com/media/cnickol26/BigDataBowl2023/main/nfl-big-data-bowl-2023/plays.csv')

In [47]:
plays['obvious_pass'] = ((plays['down'] == 2) & (plays['yardsToGo'] > 10)) | ((plays['down'] == 3) & (plays['yardsToGo'] > 4)) | ((plays['down'] == 4) & (plays['yardsToGo'] > 2))

In [48]:
plays['obvious_pass'] = plays['obvious_pass'].astype(int)

In [49]:
plays = plays[['gameId','playId','quarter','obvious_pass']]

In [50]:
plays

gameId  playId  quarter  obvious_pass
0     2021090900      97        1             0
1     2021090900     137        1             0
2     2021090900     187        1             0
3     2021090900     282        1             0
4     2021090900     349        1             1
...          ...     ...      ...           ...
8553  2021110100    4310        4             1
8554  2021110100    4363        4             0
8555  2021110100    4392        4             0
8556  2021110100    4411        4             1
8557  2021110100    4433        4             1

[8558 rows x 4 columns]

In [51]:
model_data7 = model_data6.merge(plays, on = ['gameId','playId'], how = 'left')

In [52]:
model_data7['tackle_end_engaged'] = (model_data7['tackle_end_facing'] <= 40) & (model_data7['tackle_end_dist'] <= 1)

In [53]:
model_data7['tackle_end_engaged'] = model_data7['tackle_end_engaged'].astype(int)

In [54]:
model_data7['tackle_end_engaged'].value_counts()

0    217991
1     70416
Name: tackle_end_engaged, dtype: int64

In [55]:
def dist(x_1, y_1, x_2, y_2):
    return np.sqrt(np.sum([(x_1-x_2)**2, (y_1-y_2)**2], axis=0))

In [56]:
model_data7['tackle_qb_dist'] = dist(model_data7['Tackle_new_x'], model_data7['Tackle_new_y'], model_data7['QB_new_x'], model_data7['QB_new_y'])

In [57]:
subset = model_data7[model_data7['tackle_end_engaged'] == 1]

In [58]:
subset2 = subset.groupby(['uniqueplayId','QB_Flip']).first().reset_index()

In [59]:
distance_when_engaged = subset2[['uniqueplayId','QB_Flip','tackle_qb_dist']]

In [60]:
end_subset = model_data7[model_data7['event'].isin(['pass_forward','qb_sack','qb_strip_sack'])]

In [61]:
end_distance = end_subset.groupby(['uniqueplayId','QB_Flip']).last().reset_index()

In [62]:
end_distance = end_distance[['uniqueplayId','QB_Flip','tackle_qb_dist']]

In [63]:
end_distance = end_distance.rename(columns = {'tackle_qb_dist': 'ending_distance'})

In [64]:
distance = distance_when_engaged.merge(end_distance, on = ['uniqueplayId', 'QB_Flip'], how = 'left')
distance

uniqueplayId  QB_Flip  tackle_qb_dist  ending_distance
0       202109120163      0.0        3.927467         2.871115
1       202109120163      1.0        5.853136         3.819332
2       202109120288      0.0        4.539648         4.306948
3       202109120578      1.0        4.926033         3.420322
4       202109120676      0.0        4.894865         4.512704
...              ...      ...             ...              ...
7039  20211025003684      1.0        4.883575         2.564995
7040  20211025003904      0.0        3.735305         2.465218
7041  20211025003904      1.0        4.439279         3.892364
7042  20211025003926      0.0        4.787118         7.646829
7043  20211025003926      1.0        4.256113         2.030271

[7044 rows x 4 columns]

In [65]:
distance['pct_given_up'] = (distance['tackle_qb_dist'] - distance['ending_distance']) / distance['tackle_qb_dist']
distance

uniqueplayId  QB_Flip  tackle_qb_dist  ending_distance  pct_given_up
0       202109120163      0.0        3.927467         2.871115      0.268965
1       202109120163      1.0        5.853136         3.819332      0.347472
2       202109120288      0.0        4.539648         4.306948      0.051259
3       202109120578      1.0        4.926033         3.420322      0.305664
4       202109120676      0.0        4.894865         4.512704      0.078074
...              ...      ...             ...              ...           ...
7039  20211025003684      1.0        4.883575         2.564995      0.474771
7040  20211025003904      0.0        3.735305         2.465218      0.340022
7041  20211025003904      1.0        4.439279         3.892364      0.123199
7042  20211025003926      0.0        4.787118         7.646829     -0.597376
7043  20211025003926      1.0        4.256113         2.030271      0.522975

[7044 rows x 5 columns]

In [66]:
distance = distance.rename(columns = {'tackle_qb_dist' : 'starting_distance'})

In [67]:
model_data8 = model_data7.merge(distance, on = ['uniqueplayId','QB_Flip'], how = 'left')
model_data8

uniqueplayId  QB_Flip rushType  Power_prob  Speed_prob  football_x  \
0         202109090097      1.0    Speed     0.07851     0.92149       41.56   
1         202109090097      1.0    Speed     0.07851     0.92149       41.56   
2         202109090097      1.0    Speed     0.07851     0.92149       41.56   
3         202109090097      1.0    Speed     0.07851     0.92149       41.56   
4         202109090097      1.0    Speed     0.07851     0.92149       41.56   
...                ...      ...      ...         ...         ...         ...   
288402  20211021002175      1.0    Power         NaN         NaN       92.41   
288403  20211021002175      1.0    Power         NaN         NaN       92.41   
288404  20211021002175      1.0    Power         NaN         NaN       92.41   
288405  20211021002175      1.0    Power         NaN         NaN       92.41   
288406  20211021002175      1.0    Power         NaN         NaN       92.41   

        football_y      gameId  playId  frameId  ... end_height end_weight  \
0            23.92  2021090900      97        6  ...        6-3        245   
1            23.92  2021090900      97        7  ...        6-3        245   
2            23.92  2021090900      97        8  ...        6-3        245   
3            23.92  2021090900      97        9  ...        6-3        245   
4            23.92  2021090900      97       10  ...        6-3        245   
...            ...         ...     ...      ...  ...        ...        ...   
288402       29.85  2021102100    2175       25  ...        6-4        315   
288403       29.85  2021102100    2175       26  ...        6-4        315   
288404       29.85  2021102100    2175       27  ...        6-4        315   
288405       29.85  2021102100    2175       28  ...        6-4        315   
288406       29.85  2021102100    2175       29  ...        6-4        315   

              end_name  quarter  obvious_pass  tackle_end_engaged  \
0        Micah Parsons        1             0                   0   
1        Micah Parsons        1             0                   0   
2        Micah Parsons        1             0                   0   
3        Micah Parsons        1             0                   0   
4        Micah Parsons        1             0                   0   
...                ...      ...           ...                 ...   
288402  Jordan Elliott        3             0                   0   
288403  Jordan Elliott        3             0                   0   
288404  Jordan Elliott        3             0                   0   
288405  Jordan Elliott        3             0                   0   
288406  Jordan Elliott        3             0                   0   

        tackle_qb_dist  starting_distance ending_distance pct_given_up  
0             4.554492                NaN             NaN          NaN  
1             4.610878                NaN             NaN          NaN  
2             4.651021                NaN             NaN          NaN  
3             4.667987                NaN             NaN          NaN  
4             4.714106                NaN             NaN          NaN  
...                ...                ...             ...          ...  
288402        5.114626           4.640614        5.120791    -0.103473  
288403        5.084191           4.640614        5.120791    -0.103473  
288404        5.049287           4.640614        5.120791    -0.103473  
288405        5.002799           4.640614        5.120791    -0.103473  
288406        4.936801           4.640614        5.120791    -0.103473  

[288407 rows x 65 columns]

In [68]:
## Going to need to remove any power plays where this is an NA

In [69]:
model_data8['dist_moved'] = model_data8.groupby(['uniqueplayId','QB_Flip'])['Tackle_dis'].cumsum()

In [70]:
one_yard = model_data8[model_data8['dist_moved'] >= 1].groupby(['uniqueplayId','QB_Flip']).first().reset_index()

In [71]:
one_yard = one_yard[['uniqueplayId','QB_Flip','frameId','ball_snap_frame']]

In [73]:
one_yard['tackle_get_off_time'] = (one_yard['frameId'] - one_yard['ball_snap_frame'])*.1

In [75]:
get_off = one_yard[['uniqueplayId','QB_Flip','tackle_get_off_time']]

In [76]:
model_data9 = model_data8.merge(get_off, on = ['uniqueplayId','QB_Flip'], how = 'left')

In [77]:
model_data9

uniqueplayId  QB_Flip rushType  Power_prob  Speed_prob  football_x  \
0         202109090097      1.0    Speed     0.07851     0.92149       41.56   
1         202109090097      1.0    Speed     0.07851     0.92149       41.56   
2         202109090097      1.0    Speed     0.07851     0.92149       41.56   
3         202109090097      1.0    Speed     0.07851     0.92149       41.56   
4         202109090097      1.0    Speed     0.07851     0.92149       41.56   
...                ...      ...      ...         ...         ...         ...   
288402  20211021002175      1.0    Power         NaN         NaN       92.41   
288403  20211021002175      1.0    Power         NaN         NaN       92.41   
288404  20211021002175      1.0    Power         NaN         NaN       92.41   
288405  20211021002175      1.0    Power         NaN         NaN       92.41   
288406  20211021002175      1.0    Power         NaN         NaN       92.41   

        football_y      gameId  playId  frameId  ...        end_name quarter  \
0            23.92  2021090900      97        6  ...   Micah Parsons       1   
1            23.92  2021090900      97        7  ...   Micah Parsons       1   
2            23.92  2021090900      97        8  ...   Micah Parsons       1   
3            23.92  2021090900      97        9  ...   Micah Parsons       1   
4            23.92  2021090900      97       10  ...   Micah Parsons       1   
...            ...         ...     ...      ...  ...             ...     ...   
288402       29.85  2021102100    2175       25  ...  Jordan Elliott       3   
288403       29.85  2021102100    2175       26  ...  Jordan Elliott       3   
288404       29.85  2021102100    2175       27  ...  Jordan Elliott       3   
288405       29.85  2021102100    2175       28  ...  Jordan Elliott       3   
288406       29.85  2021102100    2175       29  ...  Jordan Elliott       3   

        obvious_pass  tackle_end_engaged  tackle_qb_dist  starting_distance  \
0                  0                   0        4.554492                NaN   
1                  0                   0        4.610878                NaN   
2                  0                   0        4.651021                NaN   
3                  0                   0        4.667987                NaN   
4                  0                   0        4.714106                NaN   
...              ...                 ...             ...                ...   
288402             0                   0        5.114626           4.640614   
288403             0                   0        5.084191           4.640614   
288404             0                   0        5.049287           4.640614   
288405             0                   0        5.002799           4.640614   
288406             0                   0        4.936801           4.640614   

        ending_distance  pct_given_up dist_moved tackle_get_off_time  
0                   NaN           NaN       0.03                 0.8  
1                   NaN           NaN       0.09                 0.8  
2                   NaN           NaN       0.19                 0.8  
3                   NaN           NaN       0.34                 0.8  
4                   NaN           NaN       0.48                 0.8  
...                 ...           ...        ...                 ...  
288402         5.120791     -0.103473       3.08                 0.8  
288403         5.120791     -0.103473       3.31                 0.8  
288404         5.120791     -0.103473       3.53                 0.8  
288405         5.120791     -0.103473       3.75                 0.8  
288406         5.120791     -0.103473       3.96                 0.8  

[288407 rows x 67 columns]

In [82]:
## Now looking at yards traveled in the first second
one_sec = model_data9[model_data9['frameId'] == model_data9['ball_snap_frame'] + 10]
one_sec = one_sec[['uniqueplayId','QB_Flip','dist_moved']].rename(columns = {'dist_moved':'one_sec_dist'})

In [83]:
model_data10 = model_data9.merge(one_sec, on = ['uniqueplayId','QB_Flip'], how = 'left')

In [88]:
model_data10

uniqueplayId  QB_Flip rushType  Power_prob  Speed_prob  football_x  \
0         202109090097      1.0    Speed     0.07851     0.92149       41.56   
1         202109090097      1.0    Speed     0.07851     0.92149       41.56   
2         202109090097      1.0    Speed     0.07851     0.92149       41.56   
3         202109090097      1.0    Speed     0.07851     0.92149       41.56   
4         202109090097      1.0    Speed     0.07851     0.92149       41.56   
...                ...      ...      ...         ...         ...         ...   
288544  20211021002175      1.0    Power         NaN         NaN       92.41   
288545  20211021002175      1.0    Power         NaN         NaN       92.41   
288546  20211021002175      1.0    Power         NaN         NaN       92.41   
288547  20211021002175      1.0    Power         NaN         NaN       92.41   
288548  20211021002175      1.0    Power         NaN         NaN       92.41   

        football_y      gameId  playId  frameId  ... quarter obvious_pass  \
0            23.92  2021090900      97        6  ...       1            0   
1            23.92  2021090900      97        7  ...       1            0   
2            23.92  2021090900      97        8  ...       1            0   
3            23.92  2021090900      97        9  ...       1            0   
4            23.92  2021090900      97       10  ...       1            0   
...            ...         ...     ...      ...  ...     ...          ...   
288544       29.85  2021102100    2175       25  ...       3            0   
288545       29.85  2021102100    2175       26  ...       3            0   
288546       29.85  2021102100    2175       27  ...       3            0   
288547       29.85  2021102100    2175       28  ...       3            0   
288548       29.85  2021102100    2175       29  ...       3            0   

        tackle_end_engaged  tackle_qb_dist  starting_distance  \
0                        0        4.554492                NaN   
1                        0        4.610878                NaN   
2                        0        4.651021                NaN   
3                        0        4.667987                NaN   
4                        0        4.714106                NaN   
...                    ...             ...                ...   
288544                   0        5.114626           4.640614   
288545                   0        5.084191           4.640614   
288546                   0        5.049287           4.640614   
288547                   0        5.002799           4.640614   
288548                   0        4.936801           4.640614   

        ending_distance  pct_given_up  dist_moved tackle_get_off_time  \
0                   NaN           NaN        0.03                 0.8   
1                   NaN           NaN        0.09                 0.8   
2                   NaN           NaN        0.19                 0.8   
3                   NaN           NaN        0.34                 0.8   
4                   NaN           NaN        0.48                 0.8   
...                 ...           ...         ...                 ...   
288544         5.120791     -0.103473        3.08                 0.8   
288545         5.120791     -0.103473        3.31                 0.8   
288546         5.120791     -0.103473        3.53                 0.8   
288547         5.120791     -0.103473        3.75                 0.8   
288548         5.120791     -0.103473        3.96                 0.8   

       one_sec_dist  
0              1.46  
1              1.46  
2              1.46  
3              1.46  
4              1.46  
...             ...  
288544         1.38  
288545         1.38  
288546         1.38  
288547         1.38  
288548         1.38  

[288549 rows x 68 columns]